# 2. Eel imports

Now let's take a look at a cut of data on eel product imports. The data come from [a foreign trade database maintained by NOAA](https://www.st.nmfs.noaa.gov/commercial-fisheries/foreign-trade/).

The CSV file lives here: `../data/eels.csv`.

We'll start by importing pandas and creating a data frame.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/eels.csv')

In [ ]:
df.head()

### Check out the values

Now let's poke through the values in each column to see what we're working with using a combination of `unique()`, `min()` and `max()`. Questions we're trying to answer here: What years and months are represented? What countries are in the data? Do the numeric data make sense? Are there any obvious errors or typos to handle? Are there any holes in our data (use `info()`)?

In [ ]:
df.info()

In [ ]:
df.year.unique()

In [ ]:
df.month.unique()

In [ ]:
df.country.unique()

In [ ]:
# have to use bracket notation bc "product" is a pandas function
df['product'].unique()

**Question:** What does "ATC" stand for? _Always ask, never assume._

![atc](../img/eel-q.png "atc")

In [ ]:
print(df.kilos.max())
print(df.kilos.min())

In [ ]:
print(df.dollars.max())
print(df.dollars.min())

### Time-series data: Check for completeness

Each row in our data is one month's worth of shipments of a particular eel product from a particular country to the U.S. Which means we might want to do some time-based comparisons, so we need to check that we're dealing with complete years.

So first let's think about what we want to see: For each year that's present in our data, we want a unique list of months for those records. If we were in Excel, we might pivot to group the data by `year` and then throw `month` in the "columns" section to see what months are present for each year.

Here, we're going to do something similar:
- Select just the columns we're interested in
- Use the pandas `groupby()` method to group the records by year
- For each set of grouped data, use the pandas `unique()` method on the month column to see what months are present

When we call `groupby()` on a data frame, it returns a list of items; each item in that list is a Python data container with two elements: the _grouping_ value (year, in this case) and a dataframe of records that belong to that group (all records where year == that year).

So we can use a _for loop_ to iterate over the results and check each year.

👉For more details on _for loops_, [see this notebook](../appendix/Python%20data%20types%20and%20basic%20syntax.ipynb).

In [ ]:
for yeargroup in df[['year', 'month']].groupby('year'):
    print(yeargroup[0], yeargroup[1].month.unique())

So now we know that we have incomplete data for 2017 -- news we can use as we start our analysis.

### Come up with a list of questions

- In this data, what country ships the most eel products of any type to the U.S.?
- Same question but broken out by year.
- For each country, what was the percent change in eel shipments of all types from 2010-2016?
- What type of product is most popular?

### Q: Who ships the most eels to the U.S. (in kilos)?

We'll use our good friends `groupby()`, `sum()` and `sort_values()`to find out.

In [ ]:
df[['country', 'kilos']].groupby('country') \
                        .sum() \
                        .sort_values('kilos', ascending=False)

### Q: Who ships the most? (Broken out by year)

Now we want to create a table where the rows are countries, the columns are years, and the values are sums for that country, that year.

If we were doing this in an Excel pivot table, we'd just add "year" to the columns section. To do this in pandas, we're ... also going to use a pivot table. (Yes! Pandas has a [pivot table function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html).)

We are going to hand the `pivot_table()` function five arguments:
1. The data we're pivoting (`df`)
2. The name of the column whose values we're doing math on (`values='kilos'`)
3. The type of aggregation to apply to the values -- default is 'mean' (`aggfunc='sum'`)
4. The name of the column we're grouping on (`index='country'`)
5. The name of the column whose values will become the columns of our table (`columns='year'`)

Optionally, we can then use `sort_values()` to sort the pivot table that results by our most recent year of data.

In [ ]:
pivoted_sums = pd.pivot_table(df,
                              aggfunc='sum',
                              values='kilos',
                              index='country',
                              columns='year')

pivoted_sums.sort_values(2017, ascending=False)

### Q: What was the percent change in shipments for each country from 2010-2016?

For this question, we'll re-use the pivot table we just made and add a calculated column. First, though, we need to filter the table to include only records where the `2010` and `2016` values are not null, using the [`notnull()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.notnull.html). (Looks like just filtering for "2010 is not null" does the trick.)

In [ ]:
pivoted_sums_notnull = pivoted_sums[pivoted_sums[2010].notnull()]

pivoted_sums_notnull

Now we can add a column -- `10to16pctchange`. The syntax, and the math -- new value minus old value divided by old value -- are relatively straightforward: 

`dataframe['new_column'] = (dataframe['new_value'] - dataframe['old_value']) / dataframe['old_value'] * 100`

You might get a warning about slices vs. copies. You can ignore that for now.

In [ ]:
pivoted_sums_notnull['10to16pctchange'] = (pivoted_sums_notnull[2016] - pivoted_sums_notnull[2010]) / pivoted_sums_notnull[2010]

In [ ]:
pivoted_sums_notnull.sort_values('10to16pctchange')

### Q: What type of product is most popular (in kilos)?

We'll use `groupby()`, `sum()` and `sort_values()` again.

In [ ]:
pop_products = df[['product', 'kilos']].groupby('product') \
                                       .sum() \
                                       .sort_values('kilos', ascending=False)
pop_products

### What other questions do you want to explore?